In [1]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [2]:

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import numpy as np

sim = AerSimulator()

qc_h_then_x = QuantumCircuit(1,1)
qc_h_then_x.h(0)
qc_h_then_x.x(0)
qc_h_then_x.measure(0,0)

qc_x_then_h = QuantumCircuit(1,1)
qc_x_then_h.x(0)
qc_x_then_h.h(0)
qc_x_then_h.measure(0,0)

def get_counts(qc, shots=2048):
    qc_t = transpile(qc, sim)
    job = sim.run(qc_t, shots=shots)
    result = job.result()
    return result.get_counts()

print("Counts H then X:", get_counts(qc_h_then_x))
print("Counts X then H:", get_counts(qc_x_then_h))

qc_sv_h_then_x = QuantumCircuit(1)
qc_sv_h_then_x.h(0)
qc_sv_h_then_x.x(0)
sv_h_then_x = Statevector.from_instruction(qc_sv_h_then_x)

qc_sv_x_then_h = QuantumCircuit(1)
qc_sv_x_then_h.x(0)
qc_sv_x_then_h.h(0)
sv_x_then_h = Statevector.from_instruction(qc_sv_x_then_h)

print("Statevector H then X:", sv_h_then_x)
print("Statevector X then H:", sv_x_then_h)

Counts H then X: {'1': 1052, '0': 996}
Counts X then H: {'1': 1015, '0': 1033}
Statevector H then X: Statevector([0.70710678+0.j, 0.70710678+0.j],
            dims=(2,))
Statevector X then H: Statevector([ 0.70710678+0.j, -0.70710678+0.j],
            dims=(2,))


quantum gates generally do not commute — applying A then B is not the same as B then A. Here the non-commutation shows up as a different global/relative phase in the statevector; but because measurement probabilities in the Z-basis depend on amplitude magnitudes squared, that phase difference may not affect those probabilities.

In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

sim = AerSimulator()

thetas = [np.pi/4, np.pi/2, np.pi]
for theta in thetas:
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.rz(theta, 0)
    qc.measure(0,0)
    counts = sim.run(transpile(qc, sim), shots=2048).result().get_counts()
    print(f"theta = {theta:.6f}  counts = {counts}")


theta = 0.785398  counts = {'0': 1030, '1': 1018}
theta = 1.570796  counts = {'0': 1029, '1': 1019}
theta = 3.141593  counts = {'1': 1012, '0': 1036}


For all three angles (π/4, π/2, and π), the measurement counts are almost the same — about 50% |0⟩ and 50% |1⟩.

This happens because the RZ(θ) gate only changes the phase of the qubit, not the probability of getting 0 or 1.
So, even when θ changes, the measurement results stay nearly equal.

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

sim = AerSimulator()
theta = np.pi/2

qc_with_h = QuantumCircuit(1,1)
qc_with_h.h(0)
qc_with_h.rz(theta, 0)
qc_with_h.measure(0,0)

qc_no_h = QuantumCircuit(1,1)
qc_no_h.rz(theta, 0)
qc_no_h.measure(0,0)

counts_with_h = sim.run(transpile(qc_with_h, sim), shots=2048).result().get_counts()
counts_no_h = sim.run(transpile(qc_no_h, sim), shots=2048).result().get_counts()

print("H then RZ:", counts_with_h)
print("RZ only:", counts_no_h)


H then RZ: {'0': 1042, '1': 1006}
RZ only: {'0': 2048}
